In [1]:
import sys
import os
import pickle
import csv
import lmdb
import json

from io import BytesIO
import numpy as np

from rdflib import URIRef
from rdflib.namespace import ClosedNamespace


In [2]:
sys.path.append("/home/alireza/aida/tools/AIDA-Interchange-Format/python/aida_interchange")
from Bounding_Box import Bounding_Box
import aifutils


In [3]:
with open('../../../wsod/metadata/ont_300/300_joined_list.csv', 'r') as fin:
    all_labels = [item.strip() for item in fin]

FileNotFoundError: [Errno 2] No such file or directory: '../../../wsod/metadata/ont_300/300_joined_list.csv'

In [ ]:
with open('../../../../data/OpenImages/metadata-v4/class-names/class-descriptions.csv', 'r') as fin:
    mid2name = {item[0]: item[1] for item in csv.reader(fin)}

In [ ]:
mid2name[all_labels[0]]

In [ ]:
mid2term = {item: 'm_' + item.split('/')[-1] + '_' + mid2name[item].replace(' ', '_').replace('&', 'and') for item in all_labels}

In [ ]:
with open('../../results/det_results_postproc_11.pkl', 'rb') as fin:
    det_results_jpg = pickle.load(fin)

In [ ]:
with open('../../results/det_results_postproc_12.pkl', 'rb') as fin:
    det_results_vid = pickle.load(fin)

In [ ]:
VIS_ONTOLOGY = ClosedNamespace(
    uri=URIRef("http://www.columbia.edu/AIDA/DVMM/Ontologies/OpenImagesSubset01#"),
    terms=[mid2term[item] for item in all_labels]
)


In [ ]:
SEEDLING_ONTOLOGY = ClosedNamespace(
    uri=URIRef("https://tac.nist.gov/tracks/SM-KBP/2018/ontologies/SeedlingOntology#"),
    terms=[
        'Person',
        'Facility',
        'GeopoliticalEntity',
        'Location',
        'Organization',
        
        'Commodity',
        'Money',
        'Vehicle',
        'Weapon',
        
        'Conflict.Attack',
        'Conflict.Demonstrate',
        'Contact.Meet',
        'Existence.DamageDestroy',
        'Life.Die',
        'Life.Injure',
        'Life.Marry',
        'Movement.TransportArtifact',
        'Movement.TransportPerson',        
    ]
)


In [ ]:
mid_to_ere = {}
mid_is_event = {}
with open('../../temp/300_ere_link.csv', 'r') as fin:
    flag = False
    for row in csv.reader(fin):
        if not flag:
            flag=True
            continue
        if row[0] == '':
            continue
        if row[3] != '':
            mid_to_ere[row[0]] = SEEDLING_ONTOLOGY.term(row[3])
        if row[2] == 'No':
            mid_is_event[row[0]] = False
        elif row[2] == 'Yes':
            mid_is_event[row[0]] = True
        else:
            print(row)
            raise Exception("Neither Yes nor No")

In [4]:
root_docs = ['IC0015MEL','IC0015N8B','IC0015R7B','IC0015S3J','IC0015S43','IC0015S4O','IC0015S7Y','IC0015S83','IC0015SWP','IC0015LGD']

In [5]:
root_to_leaf = {}
leaf_to_root = {}
flag = False
with open('../../../../data/seedling/parent_children_part2.tab', 'r') as fin:
    for line in fin:
        if not flag:
            flag = True
            continue
        row = line.split()
        if root_to_leaf.get(row[0]) == None:
            root_to_leaf[row[0]] = []
        root_to_leaf[row[0]].append(row[1])
        
        leaf_to_root[row[1]] = row[0]

In [6]:
leaf_docs = []
for item in root_docs:
    leaf_docs += root_to_leaf[item]

In [7]:
vid_ids = [item.split('.')[0] for item in leaf_docs if 'mp4' in item]

In [8]:
img_ids = [item for item in leaf_docs if 'jpg' in item]

In [9]:
vid_to_shot = {}

with open('../../../../data/seedling/seedling_keyframes/msb.txt', 'r') as fin:
    for line in fin:
        row = line.split()
        shotname = row[1]
        vidname = row[1].split('_')[0]
        
        if vid_to_shot.get(vidname) == None:
            vid_to_shot[vidname] = []
        vid_to_shot[vidname].append(shotname)

In [18]:
lmdb_path = '/home/alireza/aida/export/Hassan/Filtered_all/instance_features_v2.lmdb'
env = lmdb.open(lmdb_path, map_size=1e10, readonly=True, lock=False)
txn = env.begin(write=False)
# cursor = txn.cursor()

In [19]:
lmdb_path2 = '/home/alireza/aida/export/Hassan/Filtered_all/instance_features_keyframe.lmdb'
env2 = lmdb.open(lmdb_path2, map_size=1e10, readonly=True, lock=False)
txn2 = env2.begin(write=False)
# cursor2 = txn2.cursor()

In [20]:
g = aifutils.make_graph()

system_co = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/FasterRCNN-NASNet/COCO")
system_oi = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/FasterRCNN-InceptionResNet/OpenImages")
system_ws = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/MITWeaklySupervised-ResNet/OpenImages")


In [21]:
for imgid in img_ids:
    det = det_results_jpg[imgid]
    for ii in range(det['num_detections']):
        label = det['detection_classes'][ii]
        score = det['detection_scores'][ii]
        bbox = det['detection_boxes'][ii]
        model = det['detection_models'][ii]
        
        if label not in mid_is_event:
            continue
        
        if score < 0.35:
            continue
        
        if model == 'coco':
            sys = system_co
        elif model == 'oi':
            sys = system_oi
        elif model == 'ws':
            sys = system_ws
        elif model == 'coco/J':
            sys = system_co
        elif model == 'oi/J':
            sys = system_oi
        elif model == 'ws/J':
            sys = system_ws

        labelrdf = VIS_ONTOLOGY.term(mid2term[label])
        
        if mid_to_ere.get(label) != None: 
            labelrdf_ere = mid_to_ere[label]
        
        if not mid_is_event[label]:            
            
            eid = f"http://www.columbia.edu/AIDA/DVMM/Entities/ObjectDetection/RUN00002/{imgid}/{ii}"
            
            entity = aifutils.make_entity(g, eid, sys)
            
            justification_bundle = [entity]

            type_assertion = aifutils.mark_type(g, eid + "/cu300", entity, labelrdf, sys, score)
            
            justification_bundle.append(type_assertion)
            
            if mid_to_ere.get(label) != None:
                type_assertion_2 = aifutils.mark_type(g, eid + "/ere", entity, labelrdf_ere, sys, score)

                justification_bundle.append(type_assertion_2)
                
            bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))
            aifutils.mark_image_justification(g, justification_bundle, imgid, bb, sys, score)
            
            aifutils.mark_private_data(g, entity, json.dumps({
                'columbia_vector_insmatch_v1.0': np.frombuffer(txn.get(eid.encode('utf-8')), dtype='float32').tolist()
            }), sys)
        
        else:            
            
            eid = f"http://www.columbia.edu/AIDA/DVMM/Events/ObjectDetection/RUN00002/{imgid}/{ii}"
            
            event = aifutils.make_event(g, eid, sys)
            
            justification_bundle = [event]

            type_assertion = aifutils.mark_type(g, eid + "/cu300", event, labelrdf, sys, score)
            
            justification_bundle.append(type_assertion)
            
            if mid_to_ere.get(label) != None:
                type_assertion_2 = aifutils.mark_type(g, eid + "/ere", event, labelrdf_ere, sys, score)

                justification_bundle.append(type_assertion_2)
                
                
            bb = Bounding_Box((0, 0), (0, 0))
            aifutils.mark_image_justification(g, justification_bundle, imgid, bb, sys, score)
        

        


In [22]:
for vidid in vid_ids:
    for keyframeid in vid_to_shot[vidid]:
        imgid = keyframeid + '.png'
        det = det_results_vid[imgid]

        for ii in range(det['num_detections']):
            label = det['detection_classes'][ii]
            score = det['detection_scores'][ii]
            bbox = det['detection_boxes'][ii]
            model = det['detection_models'][ii]

            if label not in mid_is_event:
                continue

            if score < 0.35:
                continue

            if model == 'coco':
                sys = system_co
            elif model == 'oi':
                sys = system_oi
            elif model == 'ws':
                sys = system_ws
            elif model == 'coco/J':
                sys = system_co
            elif model == 'oi/J':
                sys = system_oi
            elif model == 'ws/J':
                sys = system_ws

            labelrdf = VIS_ONTOLOGY.term(mid2term[label])

            if mid_to_ere.get(label) != None: 
                labelrdf_ere = mid_to_ere[label]

            if not mid_is_event[label]:            

                eid = f"http://www.columbia.edu/AIDA/DVMM/Entities/ObjectDetection/RUN00002/{imgid}/{ii}"

                entity = aifutils.make_entity(g, eid, sys)

                justification_bundle = [entity]

                type_assertion = aifutils.mark_type(g, eid + "/cu300", entity, labelrdf, sys, score)

                justification_bundle.append(type_assertion)

                if mid_to_ere.get(label) != None:
                    type_assertion_2 = aifutils.mark_type(g, eid + "/ere", entity, labelrdf_ere, sys, score)

                    justification_bundle.append(type_assertion_2)

                    
                bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))
                aifutils.mark_keyframe_video_justification(g, justification_bundle, vidid, keyframeid, bb, sys, score)
            
                aifutils.mark_private_data(g, entity, json.dumps({
                    'columbia_vector_insmatch_v1.0': np.frombuffer(txn2.get(eid.encode('utf-8')), dtype='float32').tolist()
                }), sys)

            else:            

                eid = f"http://www.columbia.edu/AIDA/DVMM/Events/ObjectDetection/RUN00002/{imgid}/{ii}"

                event = aifutils.make_event(g, eid, sys)

                justification_bundle = [event]

                type_assertion = aifutils.mark_type(g, eid + "/cu300", event, labelrdf, sys, score)

                justification_bundle.append(type_assertion)

                if mid_to_ere.get(label) != None:
                    type_assertion_2 = aifutils.mark_type(g, eid + "/ere", event, labelrdf_ere, sys, score)

                    justification_bundle.append(type_assertion_2)


                bb = Bounding_Box((0, 0), (0, 0))
                aifutils.mark_keyframe_video_justification(g, justification_bundle, vidid, keyframeid, bb, sys, score)





In [23]:
with open('../../results/aida_output_18', 'w') as fout:
    serialization = BytesIO()
    # need .buffer because serialize will write bytes, not str
    g.serialize(destination=serialization, format='turtle')
    fout.write(serialization.getvalue().decode('utf-8'))


In [22]:
with open('../../temp/ontology_export_4.ttl', 'w') as fout:
    fout.write(
'''
@prefix : <http://www.columbia.edu/AIDA/DVMM/Ontologies/OpenImagesSubset01#> .
@prefix aidaDomainCommon:  <http://darpa.mil/ontologies/AidaDomainOntologiesCommon#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

: rdf:type owl:Ontology ;
    rdfs:label "DVMM Ontology"^^xsd:string ;
    rdfs:comment "Extracted from openimages. m_something shows freebase ID."^^xsd:string ;
    owl:versionInfo "Version 0.1"^^xsd:string .

'''
    )
    for label in all_labels:
        if label not in mid_is_event:
            continue
        if mid_is_event[label] == False:
            fout.write(f':{mid2term[label]} rdfs:subClassOf aidaDomainCommon:EntityType .\n')
        else:
            fout.write(f':{mid2term[label]} rdfs:subClassOf aidaDomainCommon:EventType .\n')
        